In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re, string
import emoji
import nltk
import torch
from datasets import Dataset , Sequence , Value , Features , ClassLabel , DatasetDict

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from transformers import AutoTokenizer

from transformers.models.bert.modeling_bert import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn
from transformers import BertConfig, BertModel
from transformers import AutoConfig

from langdetect import detect

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\romoc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\romoc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\romoc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
sample_text = "Роднее всех родных Попала я в ГКБ №8 еще в декабре 2002 г. Ехать в больницу очень боялась, но делать нечего, рожать надо. О ГКБ № 8 слышала много хорошего. Когда приехала в приемное отделение, мои колени тряслись от страха, но меня там встретили очень вежливо. Осмотр врача был аккуратным и осторожным, во всем чувствовалась забота и внимание. После того, как меня определили в палату, моим врачом стала Попова Любовь Васильевна. Я благодарна судьбе за то, что она свела меня с таким прекрасным человеком. Она огромнейшей души человек и супер-врач. Внимательна, тактична, отвечала на все, порой даже глупые вопросы. Всегда может подбодрить, улыбнуться. Расскажет зачем и какие препараты нам назначают, а также зачем и какие анализы мы сдаем. С ней я себя чувствовала как за каменной стеной. К тому же у нас собралась веселая палата. На этаже есть телевизор, работае с 17 до 22 часов. Вообщем-то этого вполне хватало, т. к. у каждого были свои процедуры, капельницы, уколы и т. д. По поводу уколов и капельниц хочется сказать отдельное спасибо Анжеле. Рука у нее легкая, да и сама она, что называется, своя девчонка. Без всяких выпендрежей и заморочек. Также хочется сказать спасибо и сестре - хозяйке (к сожалению, не помню ее имя). Сперва она показалась нам очень шумной, но потом мы не представляли себе и дня без нее. Она всегда была с нами внимательна. Всегда вовремя меняла постельное белье, пеленки, полотенца. Всегда следила за тем, чтобы были чистыми холодильники. Должна сказать, что работа в отделении патологии налажена отлично. Никто не суетится, не бегает, не кричит. Все четко знают свои обязанности и оперативно выполняют работу. Все это благодаря хорошим организаторским способностям заведующей отделением, Аландровой Татьяны Вычеславовны. За то время, что я провела в больнице на обследовании, я так всех полюбила, что если бы не Новый год, то уходить не хотелось. А мой врач - Любовь Васильевна, стала для меня роднее всех родных."

def detect_language(text):
    try:
        language = detect(text)
        if language == 'ru':
            lang = "ru"
        elif language == 'en':
            lang = "en"
        else:
            lang = "Неверный язык"
        return lang
    except Exception as e:
        return f'Ошибка при определении языка: {e}'
    return

detected_language = detect_language(sample_text)
print(f'Язык текста: {detected_language}')

Язык текста: ru


In [40]:
if detected_language == 'ru':
    df_emoji = pd.read_csv("E:/project_1_for_NLP/emoji_dataset_russian.csv")
    df_slang = pd.read_csv("E:/project_1_for_NLP/russian_slang.csv")
    emoji_dict = dict(zip(df_emoji['emoji'], df_emoji['description_ru']))
    slang_dict = dict(zip(df_slang['acronym'], df_slang['expansion']))
if detected_language == 'en':
    df_slang = pd.read_csv('slang/slang/slang.csv')
    df_emoji = pd.read_csv('emoji/emoji_df.csv')
    emoji_dict = dict(zip(df_emoji['emoji'], df_emoji['name']))
    slang_dict = dict(zip(df_slang['acronym'], df_slang['expansion']))

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def convert_to_lower(text):
    return text.lower()

def replace_emoji_with_words(text):
    for emj, desc in emoji_dict.items():
        text = text.replace(emj, f" {desc} ")
    return text

def replace_slang_with_full_form(text):
    words = text.split()
    new_words = [slang_dict.get(word.lower(), word) for word in words]
    return ' '.join(new_words)

def remove_extra_white_spaces(text):
    return re.sub(r'\s+', ' ', text).strip()

def preprocessText(text):
    text = convert_to_lower(text)
    text = replace_emoji_with_words(text)
    text = replace_slang_with_full_form(text)
    text = remove_punctuation(text)
    text = remove_extra_white_spaces(text)
    return text

def preprocessBatch(batch):
    processed_texts = []

    for text in batch["text"]:
        text = convert_to_lower(text)
        text = replace_emoji_with_words(text)
        text = replace_slang_with_full_form(text)
        text = remove_punctuation(text)
        text = remove_extra_white_spaces(text)
        processed_texts.append(text)
    batch["text"] = processed_texts
    return batch

if detected_language == 'ru':
    tokenizer = AutoTokenizer.from_pretrained('seara/rubert-tiny2-russian-sentiment')
    max_len = 512
if detected_language == 'en':
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    max_len = 256


In [41]:
class BertForClassification(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.post_init()


    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                labels=None, **kwargs):
        outputs = self.bert(input_ids, attention_mask=attention_mask,
                               token_type_ids=token_type_ids, **kwargs)

        sequence_output = self.dropout(outputs[1])
        logits = self.classifier(sequence_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [47]:
if detected_language == 'ru':
    id2label = {
    0:  'Neutral',
    1: 'Positive',
    2: 'Negative'
    }

    label2id = { v:k for (k,v) in id2label.items()}
    class_names = [ 'Neutral', 'Positive', 'Negative']
    model_name = 'seara/rubert-tiny2-russian-sentiment'
    bert_config = AutoConfig.from_pretrained(model_name, num_labels=3, id2label=id2label, label2id=label2id)
    bert_model = (BertForClassification.from_pretrained(model_name, config=bert_config).to(device))
    bert_model.load_state_dict(torch.load('bert_model_rus.pth'))
if detected_language == 'en':
    id2label = {
    0: 'Negative',
    1: 'Neutral',
    2: 'Positive'
    }
    label2id = { v:k for (k,v) in id2label.items()}
    class_names = [ 'Negative','Neutral', 'Positive']
    config = AutoConfig.from_pretrained('bert-base-uncased', num_labels=3, id2label=id2label, label2id=label2id)
    bert_model = (BertForClassification.from_pretrained('bert-base-uncased', config=config).to(device))
    bert_model.load_state_dict(torch.load('bert_model_eng.pth'))
    
bert_model.eval()

C:\Users\romoc\AppData\Local\Temp\ipykernel_19768\2457069868.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bert_model.load_state_dict(torch.load('bert_model_rus.pth')

BertForClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elem

In [48]:
def eval_model(sample_text):
    sample_text = preprocessText(sample_text)
    encoding = tokenizer.encode_plus(
        sample_text,
        max_length=max_len,
        add_special_tokens=True,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_token_type_ids=False,
        return_tensors="pt"
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    output = bert_model(input_ids, attention_mask)
    _,preds = torch.max(output.logits, dim=1)
    pd.DataFrame({
        "Text": sample_text,
        "Sentiment": class_names[preds]
    },index=[0]).T
    return class_names[preds]

In [49]:
result = eval_model(sample_text)

c:\Users\romoc\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:2834: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [50]:
print(f'Тональность текста "{sample_text}" : {result}')

Тональность текста "Роднее всех родных Попала я в ГКБ №8 еще в декабре 2002 г. Ехать в больницу очень боялась, но делать нечего, рожать надо. О ГКБ № 8 слышала много хорошего. Когда приехала в приемное отделение, мои колени тряслись от страха, но меня там встретили очень вежливо. Осмотр врача был аккуратным и осторожным, во всем чувствовалась забота и внимание. После того, как меня определили в палату, моим врачом стала Попова Любовь Васильевна. Я благодарна судьбе за то, что она свела меня с таким прекрасным человеком. Она огромнейшей души человек и супер-врач. Внимательна, тактична, отвечала на все, порой даже глупые вопросы. Всегда может подбодрить, улыбнуться. Расскажет зачем и какие препараты нам назначают, а также зачем и какие анализы мы сдаем. С ней я себя чувствовала как за каменной стеной. К тому же у нас собралась веселая палата. На этаже есть телевизор, работае с 17 до 22 часов. Вообщем-то этого вполне хватало, т. к. у каждого были свои процедуры, капельницы, уколы и т. д. 